In [1]:
from utils_nlp import Tools
tools = Tools('mhk9c')

installing package 1
installing package 2
installing package 3
installing package 4
installing package 5
installing package 6
Done Installing packages


In [2]:
import sparknlp
spark = sparknlp.start(gpu=True, memory="256G") 
# spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPU

In [16]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
from pyspark.sql import functions as F

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 3.4.2
Apache Spark version: 3.1.2


In [4]:
def split_data(_df, seed = 314, train_test = [0.7, 0.3]):
    # Doesn't this create info leakage? The test set has knowledge of train's mean and standard deviation from the above scaling?    
    train_data, test_data = _df.randomSplit(train_test, seed=seed)
    return train_data, test_data

_df = tools.load_data(spark, "russian-troll-tweets-enriched")
trainDataset, testDataset = split_data(_df)
trainDataset.count()

Done loading from /project/ds5559/team1_sp22/data//russian-troll-tweets-enriched.


1467572

In [5]:
from pyspark.sql.functions import col
trainDataset.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-----+------+
|label| count|
+-----+------+
|    1|784380|
|    0|683192|
+-----+------+



In [ ]:
trainDataset.printSchema()

In [6]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("curated_content")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [7]:
use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [8]:
preds = use_pipelineModel.transform(testDataset)

In [9]:
preds.select('label', 'curated_content', 'class.result').show(10, truncate=80)

+-----+--------------------------------------------------------------------------------+------+
|label|                                                                 curated_content|result|
+-----+--------------------------------------------------------------------------------+------+
|    0|                                                     #np GHADIMI – Absent Lights|   [1]|
|    0|                                                   '@zeleness Downtime – misery'|   [0]|
|    0|10 weekly followers. 47 unfollowers. Crowdfire doesn't miss a trick - via htt...|   [1]|
|    0|4 weekly followers. 7 unfollowers. Crowdfire doesn't miss a trick - via http:...|   [1]|
|    1|"""Don't just let your business or your job make something for you; let it ma...|   [1]|
|    1|"""I have been very consistent.."" https://t.co/Qqifn2Fe9F @TheView @megynkel...|   [1]|
|    1|"""My record proves my deep and long-lasting support for Israel."" So does th...|   [1]|
|    1|"""The presidency is not a TV sho

In [1]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [11]:
df = use_pipelineModel.transform(testDataset).select('label', 'curated_content', 'class.result').toPandas()

In [12]:
df['result'] = df['result'].apply(lambda x :x[0])
df['label'] = df['label'].apply(str)

In [13]:
df.dtypes

label              object
curated_content    object
result             object
dtype: object

In [14]:
print(classification_report(df.label, df.result))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85    292627
           1       0.86      0.88      0.87    335849

    accuracy                           0.86    628476
   macro avg       0.86      0.86      0.86    628476
weighted avg       0.86      0.86      0.86    628476



In [15]:
print(accuracy_score(df.label, df.result))

0.8621124752576073
